# V4: True two layers(two pins on the different layer)

In [12]:
from parase_input_package.generate_output import *
from parase_input_package.parase_input import *
from parase_input_package.plot_output import *
import time
filepath_out='../output/bench5.router'
netlist_file_path = '../benchmark/bench5.nl'
gridfile_path  = '../benchmark/bench5.grid'
nets,net_num = parse_netlist(netlist_file_path)
rows,columns,bend_penalty,via_penalty,layer1_grid_original,layer2_grid_original = parse_gridfile(gridfile_path)

bend_penalty: 10 
 via_penalty: 20
size of layer1: (127, 317)
size of layer2: (127, 317)


In [3]:
# plot problem 根据grid文件可视化障碍物和每个格子的cost，同时标出source和target，可以显示两层
#plot_problem("../output/bench5_problem.jpg",columns,rows,layer1_grid_original,layer2_grid_original,nets)

In [13]:

def reconstruct_path(source, target, parents):
    path = []
    current = target
    while current != source:
        path.append(current)
        current = parents[current]
    path.append(source)
    path.reverse()
    return path

def mark_path_on_grid(layer1_grid,layer2_grid, path):
    #print(path)
    if path:               # 判断是否为空 如果path为None 后面不可迭代会报错 5.22
        for cell in path:
            x, y, layer = cell
            if layer == 1:
                layer1_grid[x][y] = -1
            else:
                layer2_grid[x][y] = -1

In [14]:

def get_cell_cost(layer_grid, cell,path_tmp,bend_penalty,via_penalty):  
    x, y, layer = cell
    cell_cost = 1  # 默认的单元代价
    if layer_grid[x][y] == -1:
        cell_cost = float('inf')  # -1表示无法通过的细胞
    elif layer_grid[x][y] != 1:
        cell_cost = layer_grid[x][y]  # 非单元代价
    if len(path_tmp) >= 2:
        prev_cell = path_tmp[-2]
        prev_x, prev_y, prev_layer = prev_cell
        if layer != prev_layer:
            cell_cost += via_penalty
        elif (prev_x != x and prev_y != y):  #v4，层变化的，加穿孔cost(woods5/22) 已加(justin5/22)
            cell_cost += bend_penalty
    return cell_cost

# modified by junjun
def expand_source_to_target(rows, columns, layer1_grid,layer2_grid, source, target,bend_penalty,via_penalty):#5.22 把layer2——grid输入，计算cost加入条件判断(woods5/22)
    wavefront = {}
    visited = set()
    parents = {}
    costs = {}  # Store the cumulative costs for each cell
    
    source_tuple = (source['x'], source['y'], source['layer'])
    target_tuple = (target['x'], target['y'], target['layer'])
    
    wavefront[source_tuple] = 0
    costs[source_tuple] = 1  # Initial cost for the source cell is 0

    while wavefront:
        # get lowest cost cell on a wavefront structure
        current_cell = sorted(wavefront.items(),key=lambda s:int(s[1]))[0][0]

        if current_cell == target_tuple:
            path = reconstruct_path(source_tuple, target_tuple, parents)
            return path,costs[current_cell]

        neighbors = get_neighbors(rows, columns, current_cell,layer1_grid,layer2_grid)
        neighbors.sort(key=lambda s: ((s['x']-target_tuple[0])+(s['y']-target_tuple[1]))+((s['layer']-target_tuple[2])))
        
        for neighbor in neighbors:
            neighbor_tuple = (neighbor['x'], neighbor['y'], neighbor['layer'])

            if neighbor_tuple not in visited:
                path_tmp= reconstruct_path(source_tuple, current_cell, parents)
                # Calculate the cost to reach the neighbor cell
                if(neighbor_tuple[2]== 1):
                  cost = costs[current_cell] + get_cell_cost(layer1_grid, neighbor_tuple,path_tmp,bend_penalty,via_penalty)
                if(neighbor_tuple[2]== 2):
                  cost = costs[current_cell] + get_cell_cost(layer2_grid, neighbor_tuple,path_tmp,bend_penalty,via_penalty)
                # ignore blocks
                if cost!=np.inf:
                    if neighbor_tuple not in wavefront.keys() or costs[neighbor_tuple] > cost:
                        costs[neighbor_tuple] = cost
                        parents[neighbor_tuple] = current_cell

                    if neighbor_tuple not in wavefront.keys():
                        # add cell N to waveform, indexed by pathcost
                        cost_target = cost + abs(neighbor_tuple[0]-target_tuple[0])+abs(neighbor_tuple[1]-target_tuple[1])+abs(neighbor_tuple[2]-target_tuple[2])
                        wavefront[neighbor_tuple]=cost_target     

        visited.add(current_cell)    
        del wavefront[current_cell]                  
    return None,None


def get_neighbors(rows, columns, cell,layer1_grid,layer2_grid):##v4加两个方向如果为layer==1 ，up，==2，down(woods5/22)  已加(justin5/22)     
    x, y, layer = cell
    neighbors = []
    if y < rows - 1:
        if (layer == 1 and layer1_grid[x][y+1] !=-1) or (layer == 2 and layer2_grid[x][y+1] !=-1):
            neighbors.append({'x': x, 'y': y + 1, 'layer': layer})
    if x < columns - 1:
        if (layer == 1 and layer1_grid[x+1][y] !=-1) or (layer == 2 and layer2_grid[x+1][y] !=-1):
            neighbors.append({'x': x + 1, 'y': y, 'layer': layer})
    if y > 0:
        if (layer == 1 and layer1_grid[x][y-1] !=-1) or (layer == 2 and layer2_grid[x][y-1] !=-1):
            neighbors.append({'x': x, 'y': y - 1, 'layer': layer})
    if x > 0:
        if (layer == 1 and layer1_grid[x-1][y] !=-1) or (layer == 2 and layer2_grid[x-1][y] !=-1):
            neighbors.append({'x': x - 1, 'y': y, 'layer': layer})
    if (layer == 1 and layer2_grid[x][y] !=-1) or (layer == 2 and layer1_grid[x][y] !=-1):     
        neighbors.append({'x': x, 'y': y, 'layer': 3-layer})                                          
    return neighbors

In [15]:
def true_two_layer_router(rows, columns, layer1_grid,layer2_grid, nets,bend_penalty,via_penalty):#修改了函数名，把layer2——grid输入(woods5/22)
    routing_table = {}
    costs_table = {}
    cnt = 0
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        if pin1['layer'] == 1:
            layer1_grid[pin1['x']][pin1['y']] = -1
        if pin1['layer'] == 2:
            layer2_grid[pin1['x']][pin1['y']] = -1
        if pin2['layer'] == 1:
            layer1_grid[pin2['x']][pin2['y']] = -1
        if pin2['layer'] == 2:
            layer2_grid[pin2['x']][pin2['y']] = -1
            #这里区分不同层的pin，if语句有点多(woods5/22)
        ## 防止布线在后续的pin上，先将所有的pin标记为-1；
    
    for net in nets:#在V4版本中不能分开处理
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        if pin1['layer'] == 1:
            layer1_grid[pin1['x']][pin1['y']] = 1
        if pin1['layer'] == 2:
            layer2_grid[pin1['x']][pin1['y']] = 1
        if pin2['layer'] == 1:
            layer1_grid[pin2['x']][pin2['y']] = 1
        if pin2['layer'] == 2:
            layer2_grid[pin2['x']][pin2['y']] = 1
        cnt += 1
        print('Routing net:',net_id,'\t ('+str(cnt)+'/'+str(len(nets))+')')
        path,costs = expand_source_to_target(rows, columns, layer1_grid,layer2_grid, pin1, pin2,bend_penalty,via_penalty) 
        if path is not None:
            mark_path_on_grid(layer1_grid,layer2_grid,path)
            routing_table[net_id] = path
            costs_table[net_id]=costs
    return routing_table,costs_table

In [16]:
nets

[{'net_id': 1,
  'pin1': {'x': 56, 'y': 49, 'layer': 1},
  'pin2': {'x': 58, 'y': 31, 'layer': 1}},
 {'net_id': 2,
  'pin1': {'x': 58, 'y': 49, 'layer': 1},
  'pin2': {'x': 81, 'y': 31, 'layer': 1}},
 {'net_id': 3,
  'pin1': {'x': 10, 'y': 49, 'layer': 1},
  'pin2': {'x': 56, 'y': 67, 'layer': 1}},
 {'net_id': 4,
  'pin1': {'x': 12, 'y': 49, 'layer': 1},
  'pin2': {'x': 62, 'y': 67, 'layer': 1}},
 {'net_id': 5,
  'pin1': {'x': 56, 'y': 31, 'layer': 1},
  'pin2': {'x': 56, 'y': 13, 'layer': 1}},
 {'net_id': 6,
  'pin1': {'x': 60, 'y': 31, 'layer': 1},
  'pin2': {'x': 133, 'y': 13, 'layer': 1}},
 {'net_id': 7,
  'pin1': {'x': 62, 'y': 31, 'layer': 1},
  'pin2': {'x': 115, 'y': 13, 'layer': 1}},
 {'net_id': 8,
  'pin1': {'x': 64, 'y': 31, 'layer': 1},
  'pin2': {'x': 77, 'y': 49, 'layer': 1}},
 {'net_id': 9,
  'pin1': {'x': 104, 'y': 31, 'layer': 1},
  'pin2': {'x': 147, 'y': 13, 'layer': 1}},
 {'net_id': 10,
  'pin1': {'x': 106, 'y': 31, 'layer': 1},
  'pin2': {'x': 125, 'y': 31, 'layer'

In [7]:
layer1_grid = layer1_grid_original.copy()
layer2_grid = layer2_grid_original.copy()
nets.sort(key=lambda s: ((s["pin1"]['x']-s["pin2"]['x'])**2+(s["pin1"]['y']-s["pin2"]['y'])**2))
s_time = time.time()
routing_table,costs_table=true_two_layer_router(rows, columns, layer1_grid.T,layer2_grid.T, nets,bend_penalty,via_penalty)
routing_table = dict(sorted(routing_table.items(),key=lambda x:x[0]))
print('All routing time:'+str(time.time()-s_time)[0:5]+'s')
#plot_path('../output/bench5_v4.5.jpg',columns=columns,rows=rows,grid1=layer1_grid_original,grid2=layer2_grid_original,path_dict=routing_table) 
#generate_output_file(filepath_out,net_num,routing_table)

Routing net: 121 	 (1/128)
Routing net: 123 	 (2/128)
Routing net: 68 	 (3/128)
Routing net: 72 	 (4/128)
Routing net: 75 	 (5/128)
Routing net: 92 	 (6/128)
Routing net: 13 	 (7/128)
Routing net: 62 	 (8/128)
Routing net: 35 	 (9/128)
Routing net: 87 	 (10/128)
Routing net: 76 	 (11/128)
Routing net: 11 	 (12/128)
Routing net: 40 	 (13/128)
Routing net: 82 	 (14/128)
Routing net: 22 	 (15/128)
Routing net: 37 	 (16/128)
Routing net: 5 	 (17/128)
Routing net: 48 	 (18/128)
Routing net: 81 	 (19/128)
Routing net: 114 	 (20/128)
Routing net: 1 	 (21/128)
Routing net: 19 	 (22/128)
Routing net: 23 	 (23/128)
Routing net: 49 	 (24/128)
Routing net: 125 	 (25/128)
Routing net: 122 	 (26/128)
Routing net: 10 	 (27/128)
Routing net: 100 	 (28/128)
Routing net: 69 	 (29/128)
Routing net: 51 	 (30/128)
Routing net: 106 	 (31/128)
Routing net: 89 	 (32/128)
Routing net: 8 	 (33/128)
Routing net: 63 	 (34/128)
Routing net: 47 	 (35/128)
Routing net: 70 	 (36/128)
Routing net: 107 	 (37/128)
Routi

In [10]:
sum(costs_table.values())

13940

In [11]:
len(routing_table)

125

In [9]:
plot_path('../output/bench5_0524.jpg',columns=columns,rows=rows,grid1=layer1_grid_original,grid2=layer2_grid_original,path_dict=routing_table) 